# 📚 오픈형 바이오헬스 프로그래밍 워크북 샘플

## 주제: 심전도(ECG) 데이터를 활용한 부정맥 탐지 및 분류 시스템 개발

### 1. 학습 목표
* 심전도 데이터를 처리하여 정상 심박과 부정맥을 분류하는 머신러닝 모델을 구축할 수 있다.
* 생체신호의 기본적인 처리 방법과 머신러닝 기법을 활용할 수 있다.

### 2. 필요 기본 지식
* Python 프로그래밍 기초
* 기초 통계학 및 확률론
* 신호처리 기본 개념 (주파수 분석, 필터링)
* 머신러닝 기초 (지도학습, 분류 알고리즘)
* 의료 용어 기초 (ECG, 부정맥)

### 3. 데이터셋 소개
* 데이터셋: PhysioNet MIT-BIH Arrhythmia Database
* 구성: 48명 환자의 ECG 기록 (2채널, 30분씩 기록, 360Hz)
* 데이터 규모: 약 109,000개의 심박 데이터, 5가지 주요 부정맥 유형 포함
* 접근 방법: https://physionet.org/content/mitdb/

## 4. 단계별 실습 구성 및 코드 예시
**단계 1. 데이터 준비**

In [1]:
import wfdb
record = wfdb.rdrecord('100', pn_dir='mitdb')
annotation = wfdb.rdann('100', 'atr', pn_dir='mitdb')
wfdb.plot_wfdb(record=record, annotation=annotation)

ModuleNotFoundError: No module named 'wfdb'

**단계 2. 데이터 탐색**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

signal = record.p_signal[:,0]
fs = record.fs
times = np.arange(signal.size) / fs

plt.plot(times[:3600], signal[:3600])
plt.xlabel('Time (seconds)')
plt.ylabel('ECG amplitude')
plt.title('ECG Signal - First 10 Seconds')
plt.show()

**단계 3. 데이터 전처리**

In [2]:
from scipy.signal import butter, filtfilt

# Bandpass filter
def bandpass_filter(signal, lowcut=0.5, highcut=40, fs=360, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered_signal = filtfilt(b, a, signal)
    return filtered_signal

filtered_signal = bandpass_filter(signal)
plt.plot(times[:3600], filtered_signal[:3600])
plt.title('Filtered ECG Signal - First 10 Seconds')
plt.show()

NameError: name 'signal' is not defined

**단계 4. 머신러닝 모델 구축**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Example feature extraction (RR intervals)
rr_intervals = np.diff(annotation.sample) / fs
labels = annotation.symbol[:-1]

X_train, X_test, y_train, y_test = train_test_split(rr_intervals.reshape(-1, 1), labels, test_size=0.3, random_state=42)

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

**단계 5. 모델 평가 및 결과 해석**

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion Matrix for ECG Classification')
plt.show()